In [4]:
# ---- Importing the required libraries ----
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from utils.resources_monitor import monitor_tic, monitor_toc
from utils.energy_simulator import simulate_energy_consumption
from utils.xai_methods import *

# setting seeds
global_random_seed = 42
np.random.seed(global_random_seed)

In [ ]:
# ---- Loading the dataset ----

# loading dataset
dataset = pd.read_csv("../datasets/iris.csv")

dataset.head(10)

In [ ]:
# ---- Training and testing the ML models----

# model setup
DT = DecisionTreeClassifier(random_state=global_random_seed)
KNN = KNeighborsClassifier()

# xai setup
PI = permutation_importance_explanation
SHAP = shap_explanation

# splitting features & label
X = dataset.drop('species', axis=1)
y = dataset['species']
y, _  = pd.factorize(y)

# splitting the dataset in train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=global_random_seed, stratify=y)

# training models
DT.fit(x_train, y_train)
KNN.fit(x_train, y_train)

# testing models
y_pred_1 = DT.predict(x_test)
y_pred_2 = KNN.predict(x_test)

mcc_1 = matthews_corrcoef(y_test, y_pred_1)
mcc_2 = matthews_corrcoef(y_test, y_pred_2)

# showing the MCC values
data = {
    'Model': ['DT', 'KNN'],
    'MCC': [mcc_1, mcc_2]
}
# Creating DataFrame
df = pd.DataFrame(data)
plt.figure(figsize=(8, 6))
sns.barplot(x='Model', y='MCC', data=df, palette='viridis')
plt.xlabel('Models')
plt.ylabel('MCC')
plt.title('Comparison of Model Performance')
for i, acc in enumerate([mcc_1, mcc_2]):
    plt.text(i, acc, f'{acc:.2f}', ha='center', va='bottom')
plt.ylim(0, 1)
plt.show()

In [ ]:
# ---- Executing XAI methods and showing the feature's relevance ----

# computing features relevance for DT with Permutation Importance
monitor_tic()
f_relevance_PI_DT = PI(x_test, y_test, DT)
time_PI_DT, cpu_percent_PI_DT = monitor_toc()
energy_consumed_PI_DT = simulate_energy_consumption(time_PI_DT, cpu_percent_PI_DT)

# computing features relevance for DT with SHAP
monitor_tic()
f_relevance_SHAP_DT = SHAP(x_test, y_test, DT)
time_SHAP_DT, cpu_percent_SHAP_DT = monitor_toc()
energy_consumed_SHAP_DT = simulate_energy_consumption(time_SHAP_DT, cpu_percent_SHAP_DT)

# computing features relevance for KNN with Permutation Importance
monitor_tic()
f_relevance_PI_KNN = PI(x_test, y_test, KNN)
time_PI_KNN, cpu_percent_PI_KNN = monitor_toc()
energy_consumed_PI_KNN = simulate_energy_consumption(time_PI_KNN, cpu_percent_PI_KNN)

# computing features relevance for KNN with SHAP
monitor_tic()
f_relevance_SHAP_KNN = SHAP(x_test, y_test, KNN)
time_SHAP_KNN, cpu_percent_SHAP_KNN = monitor_toc()
energy_consumed_SHAP_KNN = simulate_energy_consumption(time_SHAP_KNN, cpu_percent_SHAP_KNN)


# Creating dataframe with the results of feature relevance
data = {
    'ML model': ['DT', 'DT', 'KNN', 'KNN'],
    'XAI method': ['PI', 'SHAP', 'PI', 'SHAP'],
    'Features Relevance': [np.round(f_relevance_PI_DT, 3), np.round(f_relevance_SHAP_DT, 3), np.round(f_relevance_PI_KNN, 3), np.round(f_relevance_SHAP_KNN, 3)]
}
df_f_relevance = pd.DataFrame(data)

# Plotting correlations
print(df_f_relevance)

print(list(dataset.columns))

In [ ]:
# ---- Computing the levels of agreement ----

# Compute correlation for each ML model separately
correlation_results = {}
for model, group in df_f_relevance.groupby('ML model'):
    # Extract 'PI' and 'SHAP' arrays for the model
    pi_array = group[group['XAI method'] == 'PI']['Features Relevance'].iloc[0]
    shap_array = group[group['XAI method'] == 'SHAP']['Features Relevance'].iloc[0]
    # Compute correlation
    correlation_results[model] = np.corrcoef(pi_array, shap_array)[0, 1]

plt.figure(figsize=(8, 6))
plt.bar(correlation_results.keys(), correlation_results.values(), color='skyblue')
plt.xlabel('ML Model')
plt.ylabel('Correlation')
plt.title('Correlation between PI and SHAP for each ML Model')
plt.ylim(0, 1)  # Set y-axis limits to the range [-1, 1]
plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xticks(rotation=45)
for model, correlation in correlation_results.items():
    plt.text(model, correlation, f'{correlation}', ha='center', va='top')

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# ---- Showing the XAI costs ----

data = {
    'ML model': ['DT', 'DT', 'KNN', 'KNN'],
    'XAI method': ['PI', 'SHAP', 'PI', 'SHAP'],
    'Execution time': [time_PI_DT, time_SHAP_DT, time_PI_KNN, time_SHAP_KNN],
    'Energy consumption': [energy_consumed_PI_DT, energy_consumed_SHAP_DT, energy_consumed_PI_KNN, energy_consumed_SHAP_KNN],
}
df_costs = pd.DataFrame(data)

# Plotting costs
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
sns.barplot(data=df_costs, x='ML model', y='Execution time', hue='XAI method')
plt.title('Execution Time')
plt.xlabel('ML Model')
plt.ylabel('Time (s)')
plt.subplot(1, 2, 2)
sns.barplot(data=df_costs, x='ML model', y='Energy consumption', hue='XAI method')
plt.title('Energy Consumption')
plt.xlabel('ML Model')
plt.ylabel('Energy (J)')
plt.tight_layout()
plt.show()
